In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json
client_id = 'W__Cnh_8tZ5ssrFM3Joi'
client_secret = '7D40cw9p9x'

In [1]:
#[CODE 1]
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:   #정상
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
        
    except Exception as e: #예외처리 오류나면 여기로
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [3]:
# 네이버 뉴스 검색 url을 만들고 [CODE 1]의 getRequestUrl(url)을 호출
# 반환받은 응답 데이터를 파이썬 json 형식으로 반환하는 부분
def getNaverSearch(node, srcText, page_start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), page_start, display) #utf-8 인코딩

    url = base + node + parameters
    responseDecode = getRequestUrl(url) #[CODE 1]

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [4]:
#[CODE 3]
# JSON 형식의 응답 데이터를 필요한 항목만 정리하여 딕셔너리 리스트인 jsonResult를 구성하고 변환하는 기능
def getPostData(post, jsonResult, cnt):
    
    #post는 getNaverSearch()로 받아온 json정보
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    
    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 
                       'org_link':org_link, 'link': org_link, 'pDate':pDate})
    return

In [5]:
def main():
    node = 'news' #크롤링할 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100) #[CODE 2]
    total = jsonResponse['total']

    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):  #정보가 안들어올 때까지
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt) #[CODE 3]
            
        # 시작 페이지 바꿔가면서 json가져오기
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100) #[CODE 2]
    
    print('전체 검색 : %d 건' %total)
    
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding = 'utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)

        outfile.write(jsonFile)
    
    print("가져온 데이터 : %d 건" %(cnt))
    print('%s_naver_%s.json SAVED' % (srcText, node))
    
    
if __name__ == '__main__':
    main()

[2023-09-26 15:50:30.295575] Url Request Success
[2023-09-26 15:50:30.825575] Url Request Success
[2023-09-26 15:50:31.182704] Url Request Success
[2023-09-26 15:50:31.500675] Url Request Success
[2023-09-26 15:50:31.832885] Url Request Success
[2023-09-26 15:50:32.197057] Url Request Success
[2023-09-26 15:50:32.634736] Url Request Success
[2023-09-26 15:50:33.046823] Url Request Success
[2023-09-26 15:50:33.369142] Url Request Success
[2023-09-26 15:50:33.761197] Url Request Success
HTTP Error 400: Bad Request
[2023-09-26 15:50:34.094786] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EC%B6%A9%EB%B6%81%EB%8C%80%ED%95%99%EA%B5%90&start=1001&display=100
전체 검색 : 202152 건
가져온 데이터 : 1000 건
충북대학교_naver_news.json SAVED


## 공공데이터포털  
-출입국관광통계조회-

In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd

#인증키
ServiceKey = "8mU%2BQlNZT1Q7tEReIu9l9lCwvC%2FAvVsHriAFVLtLuktzBojVZ%2F%2FJgMDLLNB4wh5CUQxqxaQ9wrEO8SkEasSlZA%3D%3D"

In [2]:
#[CODE 1]
def getRequestUrl(url):
    req = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [4]:
#[CODE 2]
def getTourismStatsItem(yyyymm, national_code, ed_cd):
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    parameters = "?_type=json&serviceKey=" + ServiceKey #인증키
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD=" + national_code
    parameters += "&ED_CD=" + ed_cd
    url = service_url + parameters
    retData = getRequestUrl(url) #[CODE 1]
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

In [3]:
#[CODE 3]
def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    natName=''
    dataEND = "{0}{1:0>2}".format(str(nEndYear), str(12)) #데이터 끝 초기화
    isDataEnd = 0 #데이터 끝 확인용 flag 초기화 
    
    for year in range(nStartYear, nEndYear+1): 
        for month in range(1, 13):
            if(isDataEnd == 1): 
                break #데이터 끝 flag 설정되어있으면 작업 중지.
            yyyymm = "{0}{1:0>2}".format(str(year), str(month)) 
            jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd) #[CODE 2] 
            
            if (jsonData['response']['header']['resultMsg'] == 'OK'): 
                if jsonData['response']['body']['items'] == '': 
                    isDataEnd = 1 #데이터 끝 flag 설정
                    dataEND = "{0}{1:0>2}".format(str(year), str(month-1))
                    print("데이터 없음.... \n 제공되는 통계 데이터는 %s년 %s월까지입니다." %(str(year), str(month-1))) 
                    break 
                print (json.dumps(jsonData, indent=4, sort_keys=True, ensure_ascii=False)) 
            natName = jsonData['response']['body']['items']['item']['natKorNm']
            natName = natName.replace(' ', '')
            num = jsonData['response']['body']['items']['item']['num']
            ed = jsonData['response']['body']['items']['item']['ed']
            print('[ %s_%s : %s ]' %(natName, yyyymm, num))
            print('--------------------------------------------------------------') 
            jsonResult.append({'nat_name': natName, 'nat_cd': nat_cd, 'yyyymm': yyyymm, 'visit_cnt': num})
            result.append([natName, nat_cd, yyyymm, num])
    return (jsonResult, result, natName, ed, dataEND)

In [5]:
#[CODE 0]
def main():
    jsonResult = [] #수집한 데이터를 json파일로 저장할 리스트 
    result = []     #수집한 데이터를 csv파일로 저장할 리스트 

    print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
    nat_cd = input('국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : ')
    nStartYear = int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
    ed_cd = "E" #E : 방한외래관광객, D : 해외 출국
    jsonResult, result, natName, ed, dataEND = getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear) #[CODE 3]

    #파일저장 1 : json 파일
    with open('./%s_%s_%d_%s.json' % (natName, ed, nStartYear, dataEND), 'w', encoding = 'utf8') as outfile:
        # json.dumps()를 통해json 객체로 변환한 후 JSON 파일에 저장
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)
        outfile.write(jsonFile)
        
    #파일저장 2 : csv 파일
    columns = ["입국자국가", "국가코드", "입국연월", "입국자 수"]
    result_df = pd.DataFrame(result, columns = columns)
    result_df.to_csv('./%s_%s_%d_%s.csv' % (natName, ed, nStartYear,dataEND), index=False, encoding='cp949')

if __name__ == '__main__':
    main()

<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>
[2023-10-10 14:12:51.104847] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 130,
                    "natKorNm": "일  본",
                    "num": 139632,
                    "rnum": 1,
                    "ym": 201501
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
[ 일본_201501 : 139632 ]
--------------------------------------------------------------
[2023-10-10 14:12:51.174659] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 130,
                    "natKorNm": "일